---
cdt: 2024-09-04T14:00:00
title: "Meaning of Factors, Scaling and Reconstruction"
description: "A demonstration of the meaning of the factors of the PARAFAC2 model from the point of view of a HPLC-MS decomposition and how to combine them to compare the pure components with the input data at scale."
status: closed
conclusion: "Using the Zhang et al. GC-MS peak data, observation of the following features was made: The scaled elution profiles, the pure elution profiles, A as a function of K for each component and C as a function of J. A reconstruction routine was developed and visualisations the results of reconstruction. The reconstruction routine/viz should be integrated into a pipeline."
project: parafac2
---

To scale the individual compound profiles for a given sample $k$ and wavelength $j$, multiply the elution profile by the corresponding concentration loading ($kth$ row of $A$) and a spectral loading ($j$th row of $C$), as below:

In [ ]:
%load_ext autoreload
%autoreload 2

from tensorly.decomposition import parafac2 as tl_parafac2
import numpy as np
import polars as pl
import xarray as xr
import matplotlib.pyplot as plt
from pca_analysis.notebooks.code.zhang_gcms_toy_data import get_zhang_data
from pca_analysis.notebooks.code.parafac2 import Parafac2

raw_data = get_zhang_data()

_decomp, err = tl_parafac2(
    raw_data.to_numpy(),
    3,
    return_errors=True,
    verbose=True,
    n_iter_max=500,
    nn_modes="all",
)
parafac2 = Parafac2(_decomp, raw_data)


# Individual Factors

## B

In [ ]:
def demonstration_B_scaled(A, B, C):
    B_scaled = [B[k] * a * C[39] for k, a in enumerate(A)]

    for i, k in enumerate(B_scaled):
        plt.plot(k)

    plt.xlabel("I")
    plt.ylabel("y")
    plt.title("overlay of Elution Profiles in B scaled by A")


demonstration_B_scaled(parafac2.pure.B, parafac2.pure.A, parafac2.pure.C)


In [ ]:
def display_pure_elutions(B_list):
    for i, k in enumerate(B_list):
        plt.plot(k, label=i)
        plt.title("Pure B' for all k")
        plt.xlabel("I")
        plt.ylabel("y")
        plt.figtext(
            0,
            -0.05,
            "Note that the noise is proportionately high relative to the compound peaks",
        )


display_pure_elutions(parafac2.pure.B)


## A

In [ ]:
def demonstrate_A_over_K(A):
    for i, r in enumerate(A.T):
        plt.plot(r, ".-", label=i)
        plt.xlabel("K")
        plt.ylabel("y")
        plt.title("Values of A as a function of K for each Component")
        plt.figtext(
            0,
            -0.05,
            "Each indice of K represents a sample.\nTwo of the points are compounds, one is the noise",
        )
        plt.legend()


demonstrate_A_over_K(parafac2.pure.A)


## C

In [ ]:
def demonstrate_C_as_func_of_J(C):
    for i, r in enumerate(np.flip(C.T, axis=0)):
        plt.plot(r, label=i)
        plt.xlabel("J")
        plt.ylabel("y")
        plt.legend()
        plt.title("Values of C as a Function of J")
        plt.figtext(0, -0.05, "The spectral profiles for each component over the mz J")


demonstrate_C_as_func_of_J(parafac2.pure.C)


# Reconstruction

In [ ]:
k = 10
j = 39

a = parafac2.pure.A[k]
b = parafac2.pure.B[k]
c = parafac2.pure.C[j]

scaled_b_asterisk = b * a * c

ba = np.multiply(a, b)
print(b.shape)
print(a.shape)
print(ba.shape)
print(c.shape)
plt.plot(ba)
# np.dot(ba,c.T)


In [ ]:
def display_reconstruction(A, B, C, raw_data, k=10, j=39):
    a = A[k]
    b = B[k]
    c = C[j]

    # * performs np.multiply on the args
    scaled_b = b * a * c

    plt.plot(scaled_b, label=list(range(0, b.shape[1])))
    plt.plot(raw_data.isel(sample=k, mz=j), "--", label="original")
    plt.xlabel("I")
    plt.ylabel("y")
    plt.title("Demonstration of Scaled Pure Components and Original Signal")
    plt.legend()


display_reconstruction(
    A=parafac2.pure.A,
    B=parafac2.pure.B,
    C=parafac2.pure.C,
    raw_data=raw_data,
)


As we can see in the above viz, the decomposition looks sound, and fruthermore the visualisation is very informative, containing information about the pure analytes and noise compared to the convoluted signal.


# Conclusion

Using the Zhang et al. GC-MS peak data, observation of the following features was made: The scaled elution profiles, the pure elution profiles, A as a function of K for each component and C as a function of J. A reconstruction routine was developed and visualisations the results of reconstruction. The reconstruction routine/viz should be integrated into a pipeline.